In [1]:
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt

from urllib.request import urlopen, Request
import datetime

from bs4 import BeautifulSoup


# This notebook uses selenium to grab the average archival temperature (in F) for each game's home city and puts this data in a new DataFrame column. We needed to use selenium because all of the weather websites appear to use JavaScript which BeautifulSoup wouldn't parse on its own.

In [2]:
#This is just fixing typos
nba = pd.read_csv('nba.csv')
nba['home_city'].loc[nba.home_city == 'San Fransisco']  = 'San Francisco'
nba['vis_city'].loc[nba.vis_city == 'San Fransisco']  = 'San Francisco'

/Users/kessler.363/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [3]:
#Making things more uniform
nba['home_city'].loc[nba.home_city == 'Portland, OR']  = 'Portland'
nba['vis_city'].loc[nba.vis_city == 'Portland, OR']  = 'Portland'

In [4]:
#Making things more uniform
nba['home_city'].loc[nba.home_city == 'Washington, D.C']  = 'Washington'
nba['vis_city'].loc[nba.vis_city == 'Washington, D.C']  = 'Washington'

# I want to create a dictionary of cities and states more easily create URLs

In [5]:
nba.home_city.unique()

array(['Toronto', 'Los Angeles', 'Charlotte', 'Indianapolis', 'Orlando',
       'Brooklyn', 'Miami', 'Philadelphia', 'Dallas', 'San Antonio',
       'Salt Lake City', 'Phoenix', 'Portland', 'Detroit', 'Houston',
       'San Francisco', 'Boston', 'Memphis', 'New Orleans',
       'Oklahoma City', 'Denver', 'Sacramento', 'Milwaukee', 'Atlanta',
       'New York', 'Chicago', 'Cleveland', 'Minneapolis', 'Washington'],
      dtype=object)

In [6]:
states = ['ON','CA','NC','IA','FL','NY','FL','PA','TX','TX','UT','AZ','OR','MI','TX','CA','MA',
         'TN','LA','OK','CO','CA','WI','GA','NY','IL','OH','MN','DC']

In [81]:
#this creates the dictionary
citystate = {nba.home_city.unique()[i]: states[i] for i in range(len(nba.home_city.unique()))} 
print(citystate)

{'Toronto': 'ON',
 'Los Angeles': 'CA',
 'Charlotte': 'NC',
 'Indianapolis': 'IA',
 'Orlando': 'FL',
 'Brooklyn': 'NY',
 'Miami': 'FL',
 'Philadelphia': 'PA',
 'Dallas': 'TX',
 'San Antonio': 'TX',
 'Salt Lake City': 'UT',
 'Phoenix': 'AZ',
 'Portland': 'OR',
 'Detroit': 'MI',
 'Houston': 'TX',
 'San Francisco': 'CA',
 'Boston': 'MA',
 'Memphis': 'NC',
 'New Orleans': 'NY',
 'Oklahoma City': 'TN',
 'Denver': 'LA',
 'Sacramento': 'OK',
 'Milwaukee': 'CO',
 'Atlanta': 'CA',
 'New York': 'CA',
 'Chicago': 'WI',
 'Cleveland': 'MI',
 'Minneapolis': 'GA',
 'Washington': 'NY'}

In [8]:
#just testing
citystate['Washington']

'DC'

In [9]:
#This adds the states to the dataframe for easier reference later
states = []
for n in np.arange(len(nba)):
    i = nba.iloc[n].home_city
    states.append(citystate[i])

In [11]:
nba['home_state'] = states
nba

,date,start_time,vis_team,home_team,vis_points,home_points,ot,attendance,home_initial,vis_initial,...,hp1,hp2,hp3,hp4,hp5,vis_city,home_city,vis_travel_mi,home_travel_mi,home_state
0,2019-10-22,20:00:00,New Orleans Pelicans,Toronto Raptors,122,130,1,20787,TOR,NOP,...,Kyle Lowry,Fred VanVleet,Pascal Siakam,OG Anunoby,Marc Gasol,New Orleans,Toronto,0,1113,ON
1,2019-10-22,22:30:00,Los Angeles Lakers,Los Angeles Clippers,102,112,0,19068,LAC,LAL,...,Kawhi Leonard,Patrick Beverley,Landry Shamet,Patrick Patterson,Ivica Zubac,Los Angeles,Los Angeles,0,0,CA
2,2019-10-23,19:00:00,Chicago Bulls,Charlotte Hornets,125,126,0,15424,CHO,CHI,...,P.J. Washington,Cody Zeller,Dwayne Bacon,Miles Bridges,Terry Rozier,Chicago,Charlotte,587,0,NC
3,2019-10-23,19:00:00,Detroit Pistons,Indiana Pacers,119,110,0,17923,IND,DET,...,Myles Turner,Malcolm Brogdon,Domantas Sabonis,T.J. Warren,Jeremy Lamb,Detroit,Indianapolis,0,240,IA
4,2019-10-23,19:00:00,Cleveland Cavaliers,Orlando Magic,85,94,0,18846,ORL,CLE,...,Nikola Vučević,Aaron Gordon,Evan Fournier,D.J. Augustin,Jonathan Isaac,Cleveland,Orlando,0,893,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,2020-03-10,22:30:00,Los Angeles Clippers,Golden State Warriors,131,107,0,18064,GSW,LAC,...,Andrew Wiggins,Damion Lee,Marquese Chriss,Mychal Mulder,Juan Toscano-Anderson,Los Angeles,San Francisco,0,0,CA
967,2020-03-11,19:00:00,Detroit Pistons,Philadelphia 76ers,106,124,0,20172,PHI,DET,...,Josh Richardson,Tobias Harris,Al Horford,Shake Milton,Joel Embiid,Detroit,Philadelphia,0,0,PA
968,2020-03-11,19:30:00,New York Knicks,Atlanta Hawks,136,131,1,15393,ATL,NYK,...,John Collins,Trae Young,De'Andre Hunter,Kevin Huerter,Dewayne Dedmon,New York,Atlanta,0,0,GA
969,2020-03-11,19:30:00,Charlotte Hornets,Miami Heat,109,98,0,19600,MIA,CHO,...,Duncan Robinson,Bam Adebayo,Kendrick Nunn,Derrick Jones,Solomon Hill,Charlotte,Miami,0,0,FL


# Now I am going to try and create code to create wunderground URLs for each city and date

In [71]:
#This is setting the url variables
nba['date'] = pd.to_datetime(nba.date)
i = 'Toronto'
cty = i.lower().replace(' ','-')#nba.iloc[20].home_city.lower()
st = citystate[i].lower()
dt = nba.iloc[1].date.strftime('%Y-%m-%d')

In [78]:
base_url ='https://www.wunderground.com/history/daily/ca/' #The base path for historical weather data in canada
city = cty+'/'
date = 'date/'+dt
path = base_url+city+date
path

'https://www.wunderground.com/history/daily/ca/toronto/date/2019-10-22'

In [76]:
#all together now!
path = base_url+state+city+date
path

'https://www.wunderground.com/history/daily/ca/on/toronto/date/2019-10-22'

# OK that works to create a valid path. Now to test it with selenium

In [77]:
import time
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument("headless")

browser = webdriver.Chrome(executable_path='/Users/kessler.363/pythonPrograms/chromedriver',options=options)
browser.get(path)
time.sleep(2)
soup = BeautifulSoup(browser.page_source, 'html.parser')


# It took some trial and error to find the correct entry with the average temperature as it was not clearly coded, but I found the following code gave me the desired information

In [67]:
first = soup.find_all('tr',{'ngcontent-app-root-c204':"",'class':"ng-star-inserted"})
seconds = [i.find_all('td',{'_ngcontent-app-root-c204':"",'class':"ng-star-inserted"}) for i in first]
temp = float(seconds[2][0].text)
temp

76.54

# That worked! Yay! OK now lets do it for every game in a for loop

In [83]:
options = webdriver.ChromeOptions()
options.add_argument("headless")
temps = []
for i in np.arange(len(nba)):
    try:
        name = nba.iloc[i].home_city
        cty = name.lower().replace(' ','-')
        st = citystate[name].lower()
        dt = nba.iloc[i].date.strftime('%Y-%m-%d')
        if name!='Toronto':
            base_url ='https://www.wunderground.com/history/daily/us/'
            state = st+'/'
            city = cty+'/'
            date = 'date/'+dt
            path = base_url+state+city+date

        else:
            base_url ='https://www.wunderground.com/history/daily/ca/'
            city = cty+'/'
            date = 'date/'+dt
            path = base_url+city+date

        browser = webdriver.Chrome(executable_path='/Users/kessler.363/pythonPrograms/chromedriver',options=options)
        browser.get(path)
        time.sleep(2)
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        first = soup.find_all('tr',{'ngcontent-app-root-c204':"",'class':"ng-star-inserted"})
        seconds = [i.find_all('td',{'_ngcontent-app-root-c204':"",'class':"ng-star-inserted"}) for i in first]
        temp = float(seconds[2][0].text)
        temps.append(temp)
    except:
        temps.append(np.nan)

In [85]:
temps

[57.1,
 76.54,
 57.13,
 42.54,
 76.45,
 49.76,
 83.83,
 59.28,
 68.63,
 70.38,
 50.42,
 75.92,
 48.59,
 52.63,
 73.14,
 74.92,
 57.58,
 61.72,
 46.45,
 58.63,
 46.45,
 56.1,
 59.61,
 42.1,
 76.21,
 51.0,
 46.88,
 68.58,
 68.58,
 35.41,
 41.48,
 57.08,
 56.88,
 48.25,
 71.21,
 57.26,
 72.32,
 63.76,
 64.52,
 63.0,
 57.83,
 51.29,
 nan,
 57.83,
 68.76,
 17.79,
 53.5,
 67.96,
 65.08,
 40.7,
 nan,
 83.89,
 65.39,
 58.58,
 39.93,
 81.75,
 61.08,
 59.56,
 58.4,
 48.44,
 62.45,
 58.4,
 33.73,
 23.38,
 58.88,
 55.42,
 61.46,
 59.68,
 45.35,
 33.46,
 73.87,
 57.24,
 27.63,
 47.13,
 43.0,
 58.13,
 42.58,
 40.68,
 75.04,
 46.08,
 31.52,
 41.5,
 59.38,
 46.96,
 40.08,
 78.37,
 58.48,
 57.24,
 37.37,
 62.32,
 42.79,
 42.79,
 49.5,
 52.2,
 72.0,
 55.38,
 56.5,
 35.52,
 61.92,
 26.33,
 50.96,
 nan,
 35.52,
 34.97,
 nan,
 69.08,
 nan,
 53.88,
 69.41,
 46.96,
 60.76,
 57.41,
 61.0,
 74.48,
 59.21,
 23.96,
 73.72,
 25.56,
 62.63,
 49.21,
 25.56,
 46.9,
 44.16,
 45.54,
 49.58,
 70.88,
 55.63,
 38.83,
 31

In [86]:
nba['home_temp'] = temps #add to dataframe

In [87]:
nba

,date,start_time,vis_team,home_team,vis_points,home_points,ot,attendance,home_initial,vis_initial,...,hp2,hp3,hp4,hp5,vis_city,home_city,vis_travel_mi,home_travel_mi,home_state,home_temp
0,2019-10-22,20:00:00,New Orleans Pelicans,Toronto Raptors,122,130,1,20787,TOR,NOP,...,Fred VanVleet,Pascal Siakam,OG Anunoby,Marc Gasol,New Orleans,Toronto,0,1113,ON,57.10
1,2019-10-22,22:30:00,Los Angeles Lakers,Los Angeles Clippers,102,112,0,19068,LAC,LAL,...,Patrick Beverley,Landry Shamet,Patrick Patterson,Ivica Zubac,Los Angeles,Los Angeles,0,0,CA,76.54
2,2019-10-23,19:00:00,Chicago Bulls,Charlotte Hornets,125,126,0,15424,CHO,CHI,...,Cody Zeller,Dwayne Bacon,Miles Bridges,Terry Rozier,Chicago,Charlotte,587,0,NC,57.13
3,2019-10-23,19:00:00,Detroit Pistons,Indiana Pacers,119,110,0,17923,IND,DET,...,Malcolm Brogdon,Domantas Sabonis,T.J. Warren,Jeremy Lamb,Detroit,Indianapolis,0,240,IA,42.54
4,2019-10-23,19:00:00,Cleveland Cavaliers,Orlando Magic,85,94,0,18846,ORL,CLE,...,Aaron Gordon,Evan Fournier,D.J. Augustin,Jonathan Isaac,Cleveland,Orlando,0,893,FL,76.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,2020-03-10,22:30:00,Los Angeles Clippers,Golden State Warriors,131,107,0,18064,GSW,LAC,...,Damion Lee,Marquese Chriss,Mychal Mulder,Juan Toscano-Anderson,Los Angeles,San Francisco,0,0,CA,60.21
967,2020-03-11,19:00:00,Detroit Pistons,Philadelphia 76ers,106,124,0,20172,PHI,DET,...,Tobias Harris,Al Horford,Shake Milton,Joel Embiid,Detroit,Philadelphia,0,0,PA,51.71
968,2020-03-11,19:30:00,New York Knicks,Atlanta Hawks,136,131,1,15393,ATL,NYK,...,Trae Young,De'Andre Hunter,Kevin Huerter,Dewayne Dedmon,New York,Atlanta,0,0,GA,61.79
969,2020-03-11,19:30:00,Charlotte Hornets,Miami Heat,109,98,0,19600,MIA,CHO,...,Bam Adebayo,Kendrick Nunn,Derrick Jones,Solomon Hill,Charlotte,Miami,0,0,FL,74.46


In [88]:
nba.to_csv('nba_split_temp_miles.csv',index=False) #write to csv